In [10]:
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sys
sys.path.append('modeling')
from viz import viz_interactive, viz
from modeling import exponential_modeling
from bokeh.plotting import figure, show, output_notebook, output_file, save
from functions import merge_data
import load_data
from plotly.offline import init_notebook_mode, iplot
from fit_and_predict import add_preds
import json
from functions import update_severity_index as severity_index
from functions import emerging_index
plt.style.use('dark_background')
df = load_data.load_county_level()
df = df.sort_values('tot_deaths', ascending=False)
df = add_preds(df, NUM_DAYS_LIST=[1, 2, 3, 4, 5], cached_dir='data') # adds keys like "Predicted Deaths 1-day"
important_vars = load_data.important_keys(df)
print(df.keys())
df['tot_deaths_per_capita'] = df['tot_deaths'] / df['PopulationEstimate2018']
df['tot_cases_per_capita'] = df['tot_cases'] / df['PopulationEstimate2018']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Index(['id', 'Header-FIPSStandCtyCode', 'EntityofFile',
       'SecondaryEntityOfFile', 'DateofFile', 'DateofCreation', 'FileLength',
       'StateName', 'StateNameAbbreviation', 'CountyName',
       ...
       'neighbor_cases', 'y_preds_0', 'y_preds_1', 'Predicted Deaths 1-day',
       'Predicted Deaths 2-day', 'Predicted Deaths 3-day',
       'Predicted Deaths 4-day', 'Predicted Deaths 5-day',
       'Predicted Deaths 6-day', 'Predicted Deaths 7-day'],
      dtype='object', length=7360)


## how many deaths/cases are there

In [ ]:
df[['tot_deaths', 'tot_cases', 'StateName', 'CountyName', 'Predicted Deaths 1-day']].head(10)

In [ ]:
# s = f'Predicted Deaths {2}-day' # tot_deaths
s = 'tot_deaths'
num_days = 1
nonzero = df[s] > 0
plt.figure(dpi=300, figsize=(7, 3))
plt.plot(df[s].values, '.', ms=3)
plt.ylabel(s)
plt.xlabel('Counties')
# plt.yscale('log')
plt.tight_layout()
plt.show()

In [ ]:
R, C = 1, 2
NUM_COUNTIES = 9
plt.figure(dpi=500, figsize=(8, 4))


# cs = sns.diverging_palette(20, 220, n=NUM_COUNTIES)
cs = sns.color_palette("husl", NUM_COUNTIES)
for i in range(NUM_COUNTIES):
    row = df.iloc[i]
    deaths = np.array([x for x in row['deaths'] if x > 0])
    cases = np.array([x for x in row['cases'] if x > 0])
    
    CASES_ALIGNMENT = 100
    idx_align = np.where(cases > CASES_ALIGNMENT)[0][0]
    n = cases.size
    
    DEATHS_ALIGNMENT = 10
    idx_align_deaths = np.where(deaths > DEATHS_ALIGNMENT)[0][0]
    n2 = deaths.size

    
    plt.subplot(R, C, 1)
    plt.plot(np.arange(n) - idx_align, cases, alpha=0.5, label=row['CountyName'] + ' County')#, color=cs[i])
#     plt.yscale('log')
    plt.ylabel('Cumulative confirmed cases')
    plt.xlabel(f'Days since {CASES_ALIGNMENT} cases')
    plt.legend()
    
    plt.subplot(R, C, 2)
    plt.plot(np.arange(n2) - idx_align_deaths, deaths, alpha=0.5, color=cs[i])
#     plt.yscale('log')
    plt.ylabel('Cumulative deaths')
    plt.xlabel(f'Days since {DEATHS_ALIGNMENT} deaths')
plt.tight_layout()
plt.show()

# correlations

In [ ]:
d = df[[k for k in important_vars if not 'PopMale' in k and not 'PopFmle' in k and not 'MortalityAge' in k and not 'PopTotal' in k] + 
        ['tot_cases', 'tot_cases_per_capita', 'tot_deaths', 'tot_deaths_per_capita']]

viz.corrplot(d)
plt.savefig('results/correlations_heatmap.png')
plt.show()

In [ ]:
corrs = d.corr()
keys = np.array(corrs.index)
k = np.where(keys == 'tot_deaths')[0][0]
corrs_row = corrs.iloc[k]
args = np.argsort(corrs_row)
plt.figure(dpi=300, figsize=(6, 5))
plt.barh(keys[args][:-1], corrs_row[args][:-1]) # 1 to drop outcome itself
plt.xlabel('Correlation (spearman) with tot_deaths')
plt.tight_layout()
# plt.savefig('results/correlations.png')
plt.show()

In [ ]:
ks = ['PopulationDensityperSqMile2010', "TotalM.D.'s,TotNon-FedandFed2017", 'unacast_n_grade']
R, C = 1, len(ks)
plt.figure(dpi=300, figsize=(C * 3, R * 3))

for c in range(C):
    plt.subplot(R, C, c + 1)
    if c == 0:
        plt.ylabel('tot_deaths')
    plt.loglog(d[ks[c]], d['tot_deaths'], '.')
    plt.xlabel(ks[c])

plt.tight_layout()
plt.show()

# interactive plots

In [ ]:
ks = [k for k in important_vars if not 'PopMale' in k
      and not 'PopFmle' in k
      and not 'MortalityAge' in k]

**individual states no slider**

In [ ]:
# filter by state
for state in ['NY', 'WA', 'CA']:
    d = df[df["StateNameAbbreviation"] == state]

    p = viz_interactive.plot_counties(d, 
                          variable_to_distribute='tot_cases',
                          variables_to_display=ks,
                          state=state,
                          logcolor=False)
    
#     output_file(f"results/{state.lower()}.html", mode='inline')
#     show(p)
#     save(p)

**counties slider**

In [12]:
# add lat and lon to the dataframe
county_lat_lon = pd.read_csv('data/county_pop_centers.csv', dtype={'STATEFP': str, 'COUNTYFP': str})
county_lat_lon['fips'] = (county_lat_lon['STATEFP'] + county_lat_lon['COUNTYFP']).astype(np.int64)
# join to df and rename columns
df = df.join(county_lat_lon.set_index('fips'), on='countyFIPS', how='left').rename(
    columns={'LATITUDE' : 'lat', 'LONGITUDE' : 'lon'}
)

In [14]:
# Just plot the bubbles... 
viz_interactive.plot_counties_slider(df)

Loading BokehJS ...

In [16]:
# ...or plot choropleth too. Much slower and the map is less responsive
# read in county geojson
counties_json = json.load(open("data/geojson-counties-fips.json", "r"))
viz_interactive.plot_counties_slider(df, n_past_days=1, filename="results/deaths_choropleth.html", 
                                     plot_choropleth=True, counties_json=counties_json)

Loading BokehJS ...

**political leaning**

In [ ]:
# filter by state
for state in ['NY', 'WA', 'CA']:
    d = df[df["StateNameAbbreviation"] == state]

    p = viz_interactive.plot_counties(d, 
                          variable_to_distribute='dem_to_rep_ratio',
                          variables_to_display=ks,
                          state=state,
                          logcolor=False)
    show(p)

**viz curves**

In [ ]:
df_tab = df[['tot_deaths', 'tot_cases', 'CountyName', 'StateName', 
             'PopulationDensityperSqMile2010',
             'deaths', 'cases']].head(12)        
# df_tab = df_tab.rename(columns={'PopulationEstimate2018': 'Population\n(thousands})'})
df_tab = df_tab.rename(columns={'PopulationDensityperSqMile2010': 'PopDensity'})
df_tab = df_tab.rename(columns={'tot_deaths': '#Deaths', 'tot_cases': '#Cases'})
df_tab = df_tab.rename(columns={'CountyName': 'County', 'StateName': 'State'})
print(df_tab.keys())
# df_tab['Population']
keys_table = [k for k in df_tab.keys() if not k in ['deaths', 'cases']]
viz_interactive.viz_curves(df_tab, 
               key_toggle='County',
               keys_table=keys_table,
               filename='results/county_curves.html')
print('done!')

**Emerging counties index**

In [4]:
target_days=[1,2,3,4,5]
n_days_past=5

In [5]:
emerging_index.add_emerging_index(df, target_days=target_days, n_days_past=n_days_past, min_deaths=15)
df.sort_values('emerging_index', ascending=False)[['CountyName', 'StateNameAbbreviation', 'emerging_index',
                                                   '#Deaths_4/2/2020', '#Deaths_4/3/2020',
                                                   '#Deaths_4/4/2020', '#Deaths_4/5/2020', 
                                                   '#Deaths_4/6/2020', '#Deaths_4/7/2020', 
                                                   'Predicted Deaths 1-day', 'Predicted Deaths 2-day', 
                                                   'Predicted Deaths 3-day', 'Predicted Deaths 4-day',
                                                   'Predicted Deaths 5-day']].head(10)

,CountyName,StateNameAbbreviation,emerging_index,#Deaths_4/2/2020,#Deaths_4/3/2020,#Deaths_4/4/2020,#Deaths_4/5/2020,#Deaths_4/6/2020,#Deaths_4/7/2020,Predicted Deaths 1-day,Predicted Deaths 2-day,Predicted Deaths 3-day,Predicted Deaths 4-day,Predicted Deaths 5-day
1860,Westchester,NY,7.276620,67,71,197,211,233,283,354.745325,437.438676,541.759612,673.944926,842.208491
1830,Nassau,NY,4.077687,95,143,149,381,433,500,679.129082,895.098140,1188.178039,1588.250818,2137.631252
286,New Haven,CT,2.185291,17,18,29,36,41,60,72.321642,88.265169,108.240453,133.377541,165.140571
1757,Mercer,NJ,1.601322,4,5,13,16,19,24,29.362952,35.496370,43.129744,52.706591,64.811977
1852,Suffolk,NY,1.037997,84,96,175,175,236,263,325.528273,397.954470,488.483573,602.173753,745.668979
1228,Genesee,MI,0.734104,10,11,15,18,26,33,40.232505,49.147357,60.284969,74.275139,91.937411
1102,East Baton Rouge,LA,0.636358,11,13,14,22,25,31,37.003191,44.026222,52.488782,62.726168,75.157041
280,Weld,CO,0.632759,16,16,22,24,26,27,30.558925,33.968874,37.801578,42.121583,47.004821
1094,Caddo,LA,0.579771,10,10,10,19,21,26,31.129047,36.991143,44.154312,52.957655,63.833937
3058,Milwaukee,WI,0.485794,16,19,29,34,40,49,57.459723,67.189187,78.776711,92.622237,109.217559


In [ ]:
viz_interactive.plot_emerging_hotspots_grid(df, target_days=target_days, n_days_past=n_days_past)

In [15]:
emerging_index.add_emerging_index(df, 'emerging_index_2', target_days=target_days, 
                                  n_days_past=n_days_past, min_deaths=15)
df['emerging_index_diff'] = df['emerging_index'] - df['emerging_index_2']
df['emerging_index_rank'] = df['emerging_index'].rank()
df.sort_values('emerging_index_2', ascending=False)[['CountyName', 'StateNameAbbreviation', 'emerging_index', 
                                                     'emerging_index_rank', 'emerging_index_2', 'emerging_index_diff',
                                                   '#Deaths_4/2/2020', '#Deaths_4/3/2020',
                                                   '#Deaths_4/4/2020', '#Deaths_4/5/2020', 
                                                   '#Deaths_4/6/2020', '#Deaths_4/7/2020', 
                                                   'Predicted Deaths 1-day', 'Predicted Deaths 2-day', 
                                                   'Predicted Deaths 3-day', 'Predicted Deaths 4-day',
                                                   'Predicted Deaths 5-day']].head(20)

,CountyName,StateNameAbbreviation,emerging_index,emerging_index_rank,emerging_index_2,emerging_index_diff,#Deaths_4/2/2020,#Deaths_4/3/2020,#Deaths_4/4/2020,#Deaths_4/5/2020,#Deaths_4/6/2020,#Deaths_4/7/2020,Predicted Deaths 1-day,Predicted Deaths 2-day,Predicted Deaths 3-day,Predicted Deaths 4-day,Predicted Deaths 5-day
234,Denver,CO,-2.135905,1.0,1.913218,-4.049123,11,14,14,15,15,31,33.445945,38.519918,44.478747,51.501069,59.803796
1721,Clark,NV,-1.704994,2.0,1.634273,-3.339268,34,39,41,41,41,54,59.682215,68.043612,77.845478,89.403354,103.114789
2267,Philadelphia,PA,-1.342796,3.0,1.287109,-2.629905,13,14,24,28,28,58,68.453070,84.446988,105.025855,131.692468,166.472910
196,San Diego,CA,-1.340301,4.0,1.166946,-2.507248,16,17,18,19,19,31,33.013412,36.723884,40.960250,45.811102,51.381578
2833,Henrico,VA,-1.109970,6.0,1.126156,-2.236127,16,16,16,16,28,28,32.504625,37.473219,43.239546,49.949743,57.778515
200,San Mateo,CA,-1.197779,5.0,1.060247,-2.258026,10,13,13,13,13,21,22.927919,26.201135,29.927056,34.183102,39.062011
2940,King,WA,-0.985546,7.0,1.001362,-1.986908,175,188,200,208,220,226,256.071155,291.007996,333.051117,384.032702,446.332383
1836,Orange,NY,-0.873225,10.0,0.892762,-1.765986,30,40,51,51,51,53,60.342094,68.437737,77.879971,88.950530,101.999888
1133,St. John the Baptist,LA,-0.877390,8.0,0.833972,-1.711362,14,17,23,24,24,34,38.192217,43.783611,50.229042,57.679130,66.314102
2262,Montgomery,PA,-0.874083,9.0,0.826712,-1.700795,9,11,17,18,18,30,34.682200,41.232636,49.087580,58.542077,69.963410
